# Semidefinite optimization

The idea is to solve an SDP problem using cvxpy. 

In [5]:
import cvxpy as cp
import numpy as np

## Victor's ansatz

In [1]:
import cvxpy as cp
import numpy as np

# Define symbolic parameters
w11 = cp.Variable()
w12 = cp.Variable()
w13 = cp.Variable()
w22 = cp.Variable()
w23 = cp.Variable()
w33 = cp.Variable()
w46 = cp.Variable()

Wsol = cp.bmat([
               [w11, 1/8*(-2 + np.sqrt(2)), w13, 0, 0, 0], 
               [1/8*(-2 + np.sqrt(2)), w22, 1/8*(2 - np.sqrt(2)) - 2*w46, 0, 0, 0], 
               [w13, 1/8*(2 - np.sqrt(2)) - 2*w46, w33, 0, 0, 0], 
               [0, 0, 0, 1/4 + w13 - w33, 0, w46], 
               [0, 0, 0, 0, 1/4 - w11 + w13 - w22, 1/8*(2 - np.sqrt(2)) - w46], 
               [0,0, 0, w46, 1/8*(2 - np.sqrt(2)) - w46, -w13]
])

# Define trivial objective
objective = cp.Minimize(0)

# Constraints: W is PSD
constraints = [Wsol >> 0]  # W ⪰ 0

# Optional: add other constraints here if needed
# constraints += [w11 >= 0, w22 <= 1, etc.]

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)  # Or 'SCS', 'CVXOPT', 'ECOS'

# Print results
print("Status:", prob.status)
print("Numerical values:")
print("Wsol =")
print(Wsol.value)

Status: optimal
Numerical values:
Wsol =
[[ 8.83890544e-02 -7.32233047e-02 -3.66120136e-02  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-7.32233047e-02  7.32247304e-02  3.71463214e-07  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.66120136e-02  3.71463214e-07  8.83758741e-02  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.25012112e-01
   0.00000000e+00  3.66114666e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   5.17742017e-02  3.66118381e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.66114666e-02
   3.66118381e-02  3.66120136e-02]]


In [ ]:
np.savetxt("matrix.csv", Wsol.value, delimiter=",")

## Improving precision

Comment: np.set_printoptions(precision=16) counts the ciphers after the point. Here, the maximum is 16. 

In [39]:
import cvxpy as cp
import numpy as np
from mpmath import mp

# Set the desired precision (e.g., 50 decimal places)
mp.dps = 50  # set the precision to 50 decimal places

# Define symbolic parameters with high precision
w11 = cp.Variable()
w12 = cp.Variable()
w13 = cp.Variable()
w22 = cp.Variable()
w23 = cp.Variable()
w33 = cp.Variable()
w46 = cp.Variable()

# Use mpmath's high precision numbers for any symbolic calculations
sqrt2 = mp.sqrt(2)  # sqrt(2) with arbitrary precision

Wsol = cp.bmat([
    [w11, 1/8*(-2 + sqrt2), w13, 0, 0, 0],
    [1/8*(-2 + sqrt2), w22, 1/8*(2 - sqrt2) - 2*w46, 0, 0, 0],
    [w13, 1/8*(2 - sqrt2) - 2*w46, w33, 0, 0, 0],
    [0, 0, 0, 1/4 + w13 - w33, 0, w46],
    [0, 0, 0, 0, 1/4 - w11 + w13 - w22, 1/8*(2 - sqrt2) - w46],
    [0, 0, 0, w46, 1/8*(2 - sqrt2) - w46, -w13]
])

# Define trivial objective
objective = cp.Minimize(0)

# Constraints: W is PSD
constraints = [Wsol >> 0]  # W ⪰ 0

# Solve the problem using high precision
prob = cp.Problem(objective, constraints)
prob.solve(
    solver=cp.MOSEK,
    mosek_params={
        # Tight tolerances for high precision
        "MSK_DPAR_INTPNT_TOL_REL_GAP": 1e-14,
        "MSK_DPAR_INTPNT_TOL_PFEAS": 1e-14,
        "MSK_DPAR_INTPNT_TOL_DFEAS": 1e-14,
        "MSK_DPAR_INTPNT_TOL_MU_RED": 1e-14,
        "MSK_DPAR_INTPNT_TOL_PATH": 1e-14,
        
        # Tight tolerances for conic and quadratic optimization
        "MSK_DPAR_INTPNT_QO_TOL_REL_GAP": 1e-14,
        "MSK_DPAR_INTPNT_CO_TOL_REL_GAP": 1e-14,

        # Optional: reduce internal scaling to minimize artifacts
        "MSK_IPAR_INTPNT_SCALING": 0,

        # Optional: allow more steps to reach solution
        "MSK_IPAR_INTPNT_MAX_ITERATIONS": 100000,
    }
)


# Print results
print("Status:", prob.status)
print("Numerical values:")
print("Wsol =")
np.set_printoptions(precision=20) 
print(Wsol.value)

Status: optimal
Numerical values:
Wsol =
[[ 8.8388345434757237e-02 -7.3223304703363121e-02 -3.6611652463462534e-02
   0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00]
 [-7.3223304703363121e-02  7.3223306999087065e-02  4.1070480349958416e-12
   0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00]
 [-3.6611652463462534e-02  4.1070480349958416e-12  8.8388347689338773e-02
   0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00]
 [ 0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00
   1.2499999984719869e-01  0.0000000000000000e+00  3.6611652349628036e-02]
 [ 0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00
   0.0000000000000000e+00  5.1776695102693157e-02  3.6611652353735084e-02]
 [ 0.0000000000000000e+00  0.0000000000000000e+00  0.0000000000000000e+00
   3.6611652349628036e-02  3.6611652353735084e-02  3.6611652463462534e-02]]


## My certificate, dimension 5

In [4]:
import cvxpy as cp
import numpy as np

# Define symbolic parameters
g11 = cp.Variable()
g22 = cp.Variable()
g34 = cp.Variable()
g35 = cp.Variable()

Gsol = cp.bmat([
               [g11, 2*g35, 1/8*(-2 + np.sqrt(2)), 1/8*(2 - np.sqrt(2)), 0], 
               [2*g35, g22, 0, 0, 1/8*(2 - np.sqrt(2))], 
               [1/8*(-2 + np.sqrt(2)), 0, 1/4 - g11 - g34, g34, g35], 
               [1/8*(2 - np.sqrt(2)), 0, g34, 1/4 - g22 - g34, g35], 
               [0, 1/8*(2 - np.sqrt(2)), g35, g35, g34], 
])

# Define trivial objective
objective = cp.Minimize(0)

# Constraints: W is PSD
constraints = [Gsol >> 0]  # G ⪰ 0

# Optional: add other constraints here if needed
# constraints += [w11 >= 0, w22 <= 1, etc.]

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)  # Or 'SCS', 'CVXOPT', 'ECOS'

# Print results
print("Status:", prob.status)
print("Numerical values:")
print("Gsol =")
print(Gsol.value)

Status: infeasible
Numerical values:
Gsol =
None


## My certificate, dimension 6

In [7]:
g11 = cp.Variable()
g22 = cp.Variable()
g34 = cp.Variable()
g35 = cp.Variable()

s= 1/8 * (2 - np.sqrt(2))

Gsol = cp.bmat([
               [g11,	2*g35,	-s,  s,	 0,	0],
               [2*g35,	g22,	0, 	0, 	s,	0],
               [s,	0,	1/4-g11 -g34, 	g34, 	g35, 	0],
               [s,	0,	g34,  1/4-g22 -g34, 	g35,	0],
               [0,	s,	g35,   g35,	g34,	0],
               [0,	0,	0,	0,	0,	0]
])

# Define trivial objective
objective = cp.Minimize(0)

# Constraints: G is PSD
constraints = [Gsol >> 0]  # G ⪰ 0

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)  # Or 'SCS', 'CVXOPT', 'ECOS'

# Print results
print("Status:", prob.status)
print("Numerical values:")
print("Gsol =")
print(Gsol.value)

# Individual variables
print("g11 =", g11.value)
print("g22 =", g22.value)
print("g34 =", g34.value)
print("g35 =", g35.value)

Gsol_val = Gsol.value
eigenvalues = np.linalg.eigvals(Gsol_val)
print("Eigenvalues of Gsol:")
print(np.sort(np.real(eigenvalues)))

Status: optimal
Numerical values:
Gsol =
[[ 1.19735336e-01 -7.06604383e-19 -7.32233047e-02  7.32233047e-02
   0.00000000e+00  0.00000000e+00]
 [-7.06604383e-19  1.20333104e-01  0.00000000e+00  0.00000000e+00
   7.32233047e-02  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  8.22544717e-02  4.80101927e-02
  -3.53302192e-19  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  4.80101927e-02  8.16567037e-02
  -3.53302192e-19  0.00000000e+00]
 [ 0.00000000e+00  7.32233047e-02 -3.53302192e-19 -3.53302192e-19
   4.80101927e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
g11 = 0.11973533561274077
g22 = 0.12033310353796996
g34 = 0.04801019271976855
g35 = -3.5330219153632818e-19
Eigenvalues of Gsol:
[0.         0.00250585 0.03355869 0.12504391 0.12504391 0.16583745]


Here the numerical solution is PSD!

In [2]:
np.savetxt("G6sol.csv", Gsol.value, delimiter=",")

### Other solutions

In [8]:
g11 = cp.Variable()
g22 = cp.Variable()
g34 = cp.Variable()
g35 = cp.Variable()

s= 1/8 * (2 - np.sqrt(2))

Gsol = cp.bmat([
               [g11,	2*g35,	-s,  s,	 0,	0],
               [2*g35,	g22,	0, 	0, 	s,	0],
               [s,	0,	1/4-g11 -g34, 	g34, 	g35, 	0],
               [s,	0,	g34,  1/4-g22 -g34, 	g35,	0],
               [0,	s,	g35,   g35,	g34,	0],
               [0,	0,	0,	0,	0,	0]
])

# Define trivial objective
objective = cp.Minimize(g11)

# Constraints: G is PSD
constraints = [Gsol >> 0]  # G ⪰ 0

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

print("Status:", prob.status)
print("Numerical values:")
print("Gsol =")
print(Gsol.value)

print("g11 =", g11.value)
print("g22 =", g22.value)
print("g34 =", g34.value)
print("g35 =", g35.value)

Gsol_val = Gsol.value
eigenvalues = np.linalg.eigvals(Gsol_val)
print("Eigenvalues of Gsol:")
print(np.sort(np.real(eigenvalues)))

Status: optimal
Numerical values:
Gsol =
[[ 7.32232689e-02  6.55863668e-23 -7.32233047e-02  7.32233047e-02
   0.00000000e+00  0.00000000e+00]
 [ 6.55863668e-23  1.03569209e-01  0.00000000e+00  0.00000000e+00
   7.32233047e-02  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  1.25007954e-01  5.17687769e-02
   3.27931834e-23  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  5.17687769e-02  9.46620144e-02
   3.27931834e-23  0.00000000e+00]
 [ 0.00000000e+00  7.32233047e-02  3.27931834e-23  3.27931834e-23
   5.17687769e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
g11 = 0.07322326886758576
g22 = 0.103569208690288
g34 = 0.05176877693280238
g35 = 3.2793183376571874e-23
Eigenvalues of Gsol:
[-7.01623062e-09  0.00000000e+00  2.87748455e-02  1.32059196e-01
  1.32059196e-01  1.55337993e-01]


In [9]:
g11 = cp.Variable()
g22 = cp.Variable()
g34 = cp.Variable()
g35 = cp.Variable()

s= 1/8 * (2 - np.sqrt(2))

Gsol = cp.bmat([
               [g11,	2*g35,	-s,  s,	 0,	0],
               [2*g35,	g22,	0, 	0, 	s,	0],
               [s,	0,	1/4-g11 -g34, 	g34, 	g35, 	0],
               [s,	0,	g34,  1/4-g22 -g34, 	g35,	0],
               [0,	s,	g35,   g35,	g34,	0],
               [0,	0,	0,	0,	0,	0]
])

# Define trivial objective
objective = cp.Minimize(g22)

# Constraints: G is PSD
constraints = [Gsol >> 0]  # G ⪰ 0


prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)


print("Status:", prob.status)
print("Numerical values:")
print("Gsol =")
print(Gsol.value)
print("g11 =", g11.value)
print("g22 =", g22.value)
print("g34 =", g34.value)
print("g35 =", g35.value)

Gsol_val = Gsol.value
eigenvalues = np.linalg.eigvals(Gsol_val)
print("Eigenvalues of Gsol:")
print(np.sort(np.real(eigenvalues)))

Status: optimal
Numerical values:
Gsol =
[[ 9.85390105e-02 -2.74992848e-24 -7.32233047e-02  7.32233047e-02
   0.00000000e+00  0.00000000e+00]
 [-2.74992848e-24  8.30367589e-02  0.00000000e+00  0.00000000e+00
   7.32233047e-02  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  8.68913721e-02  6.45696175e-02
  -1.37496424e-24  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  6.45696175e-02  1.02393624e-01
  -1.37496424e-24  0.00000000e+00]
 [ 0.00000000e+00  7.32233047e-02 -1.37496424e-24 -1.37496424e-24
   6.45696175e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
g11 = 0.09853901045016968
g22 = 0.08303675890737659
g34 = 0.0645696174665242
g35 = -1.3749642416210892e-24
Eigenvalues of Gsol:
[-4.01996345e-09  0.00000000e+00  4.21519264e-02  7.74003601e-02
  1.47606380e-01  1.68271720e-01]


In [10]:
g11 = cp.Variable()
g22 = cp.Variable()
g34 = cp.Variable()
g35 = cp.Variable()

s= 1/8 * (2 - np.sqrt(2))

Gsol = cp.bmat([
               [g11,	2*g35,	-s,  s,	 0,	0],
               [2*g35,	g22,	0, 	0, 	s,	0],
               [s,	0,	1/4-g11 -g34, 	g34, 	g35, 	0],
               [s,	0,	g34,  1/4-g22 -g34, 	g35,	0],
               [0,	s,	g35,   g35,	g34,	0],
               [0,	0,	0,	0,	0,	0]
])

# Define trivial objective
objective = cp.Minimize(g34)

# Constraints: G is PSD
constraints = [Gsol >> 0]  # G ⪰ 0

prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)

print("Status:", prob.status)
print("Numerical values:")
print("Gsol =")
print(Gsol.value)
print("g11 =", g11.value)
print("g22 =", g22.value)
print("g34 =", g34.value)
print("g35 =", g35.value)

Gsol_val = Gsol.value
eigenvalues = np.linalg.eigvals(Gsol_val)
print("Eigenvalues of Gsol:")
print(np.sort(np.real(eigenvalues)))

Status: optimal
Numerical values:
Gsol =
[[ 1.51468897e-01  9.81084204e-24 -7.32233047e-02  7.32233047e-02
   0.00000000e+00  0.00000000e+00]
 [ 9.81084204e-24  1.66963229e-01  0.00000000e+00  0.00000000e+00
   7.32233047e-02  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  6.64183315e-02  3.21127716e-02
   4.90542102e-24  0.00000000e+00]
 [ 7.32233047e-02  0.00000000e+00  3.21127716e-02  5.09239993e-02
   4.90542102e-24  0.00000000e+00]
 [ 0.00000000e+00  7.32233047e-02  4.90542102e-24  4.90542102e-24
   3.21127716e-02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
g11 = 0.15146889689839627
g22 = 0.1669632290259348
g34 = 0.03211277163416484
g35 = 4.905421020262089e-24
Eigenvalues of Gsol:
[-1.54046166e-09  0.00000000e+00  1.73130601e-02  1.19247961e-01
  1.32250207e-01  1.99076002e-01]


In [11]:
g11 = cp.Variable()
g22 = cp.Variable()
g34 = cp.Variable()

s= 1/8 * (2 - np.sqrt(2))

Gsol = cp.bmat([
               [g11,	0,	-s,  s,	 0,	0],
               [0,	g22,	0, 	0, 	s,	0],
               [s,	0,	1/4-g11 -g34, 	g34, 	0, 	0],
               [s,	0,	g34,  1/4-g22 -g34, 	0,	0],
               [0,	s,	0,   0,	g34,	0],
               [0,	0,	0,	0,	0,	0]
])

# Define trivial objective
objective = cp.Minimize(cp.abs(g11 - 1))

# Constraints: G is PSD
constraints = [Gsol >> 0]  # G ⪰ 0

# Solve the problem
prob = cp.Problem(objective, constraints)
prob.solve(solver=cp.MOSEK)


print("Status:", prob.status)
print("Numerical values:")
print("Gsol =")
print(Gsol.value)
print("g11 =", g11.value)
print("g22 =", g22.value)
print("g34 =", g34.value)

Gsol_val = Gsol.value
eigenvalues = np.linalg.eigvals(Gsol_val)
print("Eigenvalues of Gsol:")
print(np.sort(np.real(eigenvalues)))

Status: optimal
Numerical values:
Gsol =
[[ 0.17677672  0.         -0.0732233   0.0732233   0.          0.        ]
 [ 0.          0.14645546  0.          0.          0.0732233   0.        ]
 [ 0.0732233   0.          0.03661385  0.03660944  0.          0.        ]
 [ 0.0732233   0.          0.03660944  0.06693511  0.          0.        ]
 [ 0.          0.0732233   0.          0.          0.03660944  0.        ]
 [ 0.          0.          0.          0.          0.          0.        ]]
g11 = 0.17677671588207342
g22 = 0.14645545741251023
g34 = 0.036609436782452806
Eigenvalues of Gsol:
[-2.96120700e-09  0.00000000e+00  3.03266188e-02  6.34287332e-02
  1.83064897e-01  1.86570317e-01]
